In [1]:
# load and optionally install required packages
if (!require('pacman')) install.packages('pacman')
pacman::p_load(
    dplyr,
    readr,
    scales
)

Loading required package: pacman



In [2]:
csv_files <- list.files(path='./data/generated', full.names = T, pattern = 'filtered*')
data_groups <- lapply(csv_files, read_csv)
data.raw <- do.call(rbind, data_groups)

Rows: 503 Columns: 50
── Column specification ─────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (30): content, domain_url, images.url, lang, matched_profile, noise_cate...
dbl (19): engagement, fluency_level, porn_level, reach, sentiment, url_views...
lgl  (1): title

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 7701 Columns: 50
── Column specification ─────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (30): content, domain_url, images.url, lang, matched_profile, noise_cate...
dbl (19): engagement, fluency_level, porn_level, reach, sentiment, url_views...
lgl  (1): title

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 42048 Columns: 50
── 


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 30857 Columns: 50
── Column specification ─────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (30): content, domain_url, images.url, lang, matched_profile, noise_cate...
dbl (19): engagement, fluency_level, porn_level, reach, sentiment, url_views...
lgl  (1): title

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 62654 Columns: 50
── Column specification ─────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (31): content, domain_url, images.url, lang, matched_profile, noise_cate...
dbl (19): engagement, fluency_level, porn_level, reach, sentiment, url_views...

ℹ Use `spec()` to retrieve the full col

In [3]:
bots <- read_csv('./data/generated/botometer_usernames_by_cutoff.csv')

Rows: 80363 Columns: 1
── Column specification ─────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (1): username

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [4]:
data.raw %>% nrow()

[1] 646167

In [5]:
data <- anti_join(data.raw, bots)

Joining, by = "username"


In [6]:
data %>% nrow()

[1] 558120

In [7]:
percent((data %>% nrow()) / (data.raw %>% nrow()))

[1] "86%"

In [8]:
colnames(data)

[1] "content"                    "domain_url"                
 [3] "engagement"                 "fluency_level"             
 [5] "images.url"                 "lang"                      
 [7] "matched_profile"            "noise_category"            
 [9] "parent_url"                 "porn_level"                
[11] "post_type"                  "published"                 
[13] "reach"                      "sentiment"                 
[15] "tags_internal"              "title"                     
[17] "url"                        "url_views"                 
[19] "videos.url"                 "word_count"                
[21] "username"                   "facebook_followers"        
[23] "facebook_likes"             "facebook_shares"           
[25] "twitter_followers"          "twitter_likes"             
[27] "twitter_retweets"           "twitter_shares"            
[29] "instagram_followers"        "instagram_likes"           
[31] "author_name"                "author_birthday"           
[33] "author_birthday_resolution" "author_gender"             
[35] "author_short_name"          "author_url"                
[37] "author_description"         "author_continent"          
[39] "author_country"             "author_country_code"       
[41] "author_region"              "author_city"               
[43] "article_city"               "article_latitude"          
[45] "article_longitide"          "source_continent"          
[47] "source_country"             "source_country_code"       
[49] "source_region"              "source_city"

In [9]:
data %>% filter(matched_profile == 'SEARCH / Food Access / Food Quality (Sp)')

content,domain_url,engagement,fluency_level,images.url,lang,matched_profile,noise_category,parent_url,porn_level,⋯,author_region,author_city,article_city,article_latitude,article_longitide,source_continent,source_country,source_country_code,source_region,source_city
<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,⋯,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
"The food! Mole, salsa verde, hot chocolate, queso fresco, carne asada. One of my very best friends is Mexican and a chef. Mexican food is so delicious.",http://twitter.com/,0,100,NA,en,SEARCH / Food Access / Food Quality (Sp),NA,http://twitter.com/sftromin/status/1272057256348016640,0,⋯,New York,New York City,New York City,40.75104,-73.97644,North America,United States,us,New York,New York City


In [10]:
data %>% distinct(matched_profile)

matched_profile
<chr>
SEARCH / Food Access / Access to Culturally Appropriate Foods (Eng)
SEARCH / Food Access / Access to Culturally Appropriate Foods (Sp)
SEARCH / Food Access / Community Fridges
SEARCH / Food Access / Community Gardens
SEARCH / Food Access / EBT/SNAP
SEARCH / Food Access / Food Insecurity
SEARCH / Food Access / Food Justice (Eng)
SEARCH / Food Access / Food Pantries
SEARCH / Food Access / Food Prices


In [11]:
rekey <- function(old_key){
    return(switch(
        old_key,
        'SEARCH / Food Access / Access to Culturally Appropriate Foods (Eng)' = 'access_justice',
        'SEARCH / Food Access / Access to Culturally Appropriate Foods (Sp)' = 'access_justice',
        'SEARCH / Food Access / Community Fridges' = 'community',
        'SEARCH / Food Access / Community Gardens' = 'community',
        'SEARCH / Food Access / EBT/SNAP' = 'snap_wic_school',
        'SEARCH / Food Access / Food Insecurity' = 'food_insecurity',
        'SEARCH / Food Access / Food Justice (Eng)' = 'access_justice',
        'SEARCH / Food Access / Food Pantries' = 'community',
        'SEARCH / Food Access / Food Prices' = 'prices',
        'SEARCH / Food Access / Food Quality (Eng)' = 'food_quality',
        'SEARCH / Food Access / Food Quality (Sp)' = 'food_quality',
        'SEARCH / Food Access / School Food' = 'snap_wic_school',
        'SEARCH / Food Access / Grocery Stores/Brands' = 'stores_brands',
        'SEARCH / Food Access / Struggle (Eng)' = 'struggle',
        'SEARCH / Food Access / WIC/eWIC' = 'snap_wic_school',
    ))
}
data.rekeyed <- data %>% rowwise() %>% mutate(cluster = rekey(matched_profile))

In [38]:
data.rekeyed

content,domain_url,engagement,fluency_level,images.url,lang,matched_profile,noise_category,parent_url,porn_level,⋯,author_city,article_city,article_latitude,article_longitide,source_continent,source_country,source_country_code,source_region,source_city,cluster
<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,⋯,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
"RT : Just gave an end of year donation to Kosher Soup Kitchens, doing the vital work of feeding the needy. Due to the crises this year, demand for their services increased by 500%! You have until midnight to make a tax deductible donation for 2020.",http://twitter.com/,0,100,NA,en,SEARCH / Food Access / Access to Culturally Appropriate Foods (Eng),NA,http://twitter.com/YeahThatsKosher/status/1344819866096832512,0,⋯,New York City,New York City,40.75104,-73.97644,North America,United States,us,New York,New York City,access_justice
"What happened in LES wasn’t acceptable. Racism isn’t acceptable towards any group. In Chinatown, Chinese grocers were arrested after being attacked & racially abused by black shoppers who didn’t wear masks & refused to leave. At end, they still didn’t need to wear masks. #APAHM",http://twitter.com/,0,85,http://pbs.twimg.com/ext_tw_video_thumb/1257658056986542091/pu/img/cX2H10p8JWrXDvTn.jpg,en,SEARCH / Food Access / Access to Culturally Appropriate Foods (Eng),NA,http://twitter.com/CriticalRace/status/1257495697248661505,0,⋯,New York City,New York City,40.75104,-73.97644,North America,United States,us,New York,New York City,access_justice
"RT : The alleged accomplices of the terror attacks on Charlie Hebdo and a kosher grocery store went on trial Wednesday, a day after the satirical magazine republished cartoons of the Prophet Muhammad that triggered the attackers five years ago",http://twitter.com/,0,100,NA,en,SEARCH / Food Access / Access to Culturally Appropriate Foods (Eng),NA,http://twitter.com/WSJ/status/1301149728395194369,0,⋯,New York City,New York City,40.75104,-73.97644,North America,United States,us,New York,New York City,access_justice
"RT : The alleged accomplices of the terror attacks on Charlie Hebdo and a kosher grocery store went on trial Wednesday, a day after the satirical magazine republished cartoons of the Prophet Muhammad that triggered the attackers five years ago",http://twitter.com/,0,100,NA,en,SEARCH / Food Access / Access to Culturally Appropriate Foods (Eng),NA,http://twitter.com/WSJ/status/1301149728395194369,0,⋯,New York City,New York City,40.75104,-73.97644,North America,United States,us,New York,New York City,access_justice
"RT : . #fallfest finale-2 session webinar on #Bronx #communityservices w essential services partner ⁦⁩ 40 viewers in total attended the English & Spanish sessions covering food pantry, Snap, #financialcounseling #evictionprevention & #affordablehousing programs",http://twitter.com/,0,16,http://pbs.twimg.com/media/EphBUUHWwAExql2.jpg,en,SEARCH / Food Access / Access to Culturally Appropriate Foods (Eng),NA,http://twitter.com/UNHP/status/1339892715287425026,0,⋯,New York City,New York City,40.75104,-73.97644,North America,United States,us,New York,New York City,access_justice
"Can't really explain how happy it makes me that my Spanish grocery store finally stocks this. if I had room in my tiny fridge, I would have bought them out!",http://twitter.com/,10,100,http://pbs.twimg.com/media/Ec1YO2SXgAMqPir.jpg,en,SEARCH / Food Access / Access to Culturally Appropriate Foods (Eng),NA,http://twitter.com/DavidGMcGreevy/status/1282778497396674565,0,⋯,New York City,New York City,40.75104,-73.97644,North America,United States,us,New York,New York City,access_justice
I spent hella on groceries & stopped by that halal truck before coming home. Cause I’m definitely not about to cook after shopping. 😂,http://twitter.com/,0,100,NA,en,SEARCH / Food Access / Access to Culturally Appropriate Foods (Eng),NA,http://twitter.com/money_melle/status/1374492548882853898,0,⋯,New York City,New York City,

In [39]:
data.rekeyed %>% write_csv('./data/generated/debotted_combined.csv')